In [12]:
import pandas as pd

In [13]:
df = pd.read_csv('neurolinguistics_study-merged-0421.csv')

df.head()

,workerid,proliferate.condition,correct_answer,prime,probeContent,response,rt,slide_number_in_experiment,target,trial_type,...,subject_information.gender,subject_information.otherLanguage,subject_information.problems,subject_information.prolific_ID,system.Browser,system.OS,system.screenH,system.screenW,time_in_minutes,error
0,5,condition1,2,餵,懼,j,2233,8,懼,phon-sem,...,Male,漢語，英語，西班牙語,無,664db585355de70e17541e5a,Chrome,Windows,1080,1920,17.978567,NaN
1,5,condition1,2,眠,務,j,2181,9,務,con,...,Male,漢語，英語，西班牙語,無,664db585355de70e17541e5a,Chrome,Windows,1080,1920,17.978567,NaN
2,5,condition1,1,城,城,f,2344,10,功,phon-sem,...,Male,漢語，英語，西班牙語,無,664db585355de70e17541e5a,Chrome,Windows,1080,1920,17.978567,NaN
3,5,condition1,1,真,真,f,2201,11,實,sem-sem,...,Male,漢語，英語，西班牙語,無,664db585355de70e17541e5a,Chrome,Windows,1080,1920,17.978567,NaN
4,5,condition1,2,孩,侵,j,2392,12,侵,con,...,Male,漢語，英語，西班牙語,無,664db585355de70e17541e5a,Chrome,Windows,1080,1920,17.978567,NaN


In [14]:
df['workerid'].value_counts()

workerid
5     180
7     180
11    180
8     180
10    180
Name: count, dtype: int64

In [31]:
# Step 1: Get all columns that start with 'subject_information.'
info_cols = [col for col in df.columns if col.startswith('subject_information.')]
keywords = ['comments', 'problems', 'assess','gender']
info_cols = [col for col in info_cols if not any(k in col for k in keywords)]

# Step 2: Get unique worker IDs
subject_ids = df['workerid'].unique()

# Step 3: Loop over each subject and print their info
for workerid in subject_ids:
    df_subj = df[df['workerid'] == workerid]
    print(f'----- workerid: {workerid} -----')
    for col in info_cols:
        unique_val = df_subj[col].unique()
        print(f'{col}: {unique_val}')


----- workerid: 5 -----
subject_information.age: [31]
subject_information.chinese_country_years: [25]
subject_information.chinese_family_years: [25]
subject_information.chinese_school_work_years_spoken: [19]
subject_information.chinese_school_work_years_written: [19]
subject_information.education: [4]
subject_information.otherLanguage: ['漢語，英語，西班牙語']
subject_information.prolific_ID: ['664db585355de70e17541e5a']
----- workerid: 7 -----
subject_information.age: [28]
subject_information.chinese_country_years: [12]
subject_information.chinese_family_years: [16]
subject_information.chinese_school_work_years_spoken: [6]
subject_information.chinese_school_work_years_written: [0]
subject_information.education: [3]
subject_information.otherLanguage: ['中文，英語']
subject_information.prolific_ID: ['5faefcc22be93b11829e84df']
----- workerid: 11 -----
subject_information.age: [39]
subject_information.chinese_country_years: [37]
subject_information.chinese_family_years: [39]
subject_information.chinese

In [16]:
subjects = df['workerid'].unique().tolist()
remove_set = {11}
subjects = [x for x in subjects if x not in remove_set]
n_subjects = len(subjects)

print('Total number of subjects: ', n_subjects)

Total number of subjects:  4


In [17]:
df.columns

Index(['workerid', 'proliferate.condition', 'correct_answer', 'prime',
       'probeContent', 'response', 'rt', 'slide_number_in_experiment',
       'target', 'trial_type', 'catch_trials', 'subject_information.age',
       'subject_information.assess',
       'subject_information.chinese_country_years',
       'subject_information.chinese_family_years',
       'subject_information.chinese_school_work_years_spoken',
       'subject_information.chinese_school_work_years_written',
       'subject_information.comments', 'subject_information.education',
       'subject_information.gender', 'subject_information.otherLanguage',
       'subject_information.problems', 'subject_information.prolific_ID',
       'system.Browser', 'system.OS', 'system.screenH', 'system.screenW',
       'time_in_minutes', 'error'],
      dtype='object')

In [18]:
def compute_accuracy(df):

    ntrials = len(df)
    n_correct_trials = 0
    
    for ii in range(len(df)):

        trial = df.iloc[ii]
        correct_answer = trial['correct_answer']
        response = trial['response']

        if (correct_answer == 1 and response == 'f') or (correct_answer == 2 and response == 'j'):

            n_correct_trials += 1

    accuracy_percent = n_correct_trials / ntrials

    return accuracy_percent

In [27]:
def compute_rt(df, trial_type):

    df_trial_type = df[df['trial_type'] == trial_type]
    # df_trial_type = df_trial_type[df_trial_type['correct_answer'] == 1]


    rt = df_trial_type['rt']

    rt = rt - (300 + 100 + 500 + 800)

    assert(rt > 0).all(), "Some RTs are not greater than 0"

    return(rt.mean())
    

In [28]:
results = []

for subj in range(n_subjects):
    workerid = subjects[subj]
    df_workerid = df[df['workerid'] == workerid]

    overall_accuracy = compute_accuracy(df_workerid)

    trial_types = df_workerid['trial_type'].unique()
    for trial_type in trial_types:
        mean_rt = compute_rt(df_workerid, trial_type)

        # Append a dictionary of results for this subject and trial type
        results.append({
            'workerid': workerid,
            'accuracy': overall_accuracy,
            'trial_type': trial_type,
            'mean_rt': mean_rt
        })

# Convert the list of dictionaries into a DataFrame
results_df = pd.DataFrame(results)


In [29]:
results_df.head()

,workerid,accuracy,trial_type,mean_rt
0,5,0.972222,phon-sem,568.555556
1,5,0.972222,con,617.311111
2,5,0.972222,sem-sem,542.822222
3,5,0.972222,phon-phon,546.422222
4,7,0.988889,phon-phon,608.844444


In [30]:
trial_types = df['trial_type'].unique()

for tt in range(len(trial_types)):

    this_type = trial_types[tt]

    df_type = results_df[results_df['trial_type'] == this_type]

    mean_rt = df_type['mean_rt'].mean()

    print(this_type, mean_rt)

phon-sem 631.9166666666666
con 657.6611111111112
sem-sem 625.1888888888889
phon-phon 660.1722222222222
